## First attempt at combining the alm, wigner rotations and visibility response

In [133]:
import numpy as np
import scipy as sp

# Plotting
#import matplotlib.pyplot as plt
import cmocean
import cmocean.cm as cmo
import pylab as plt

# Mapping
import healpy as hp

# Wigner D matrices
import spherical, quaternionic

# Simulation
import pyuvsim

# Hydra
import sys
sys.path.append("/home/katgla/Documents/hera/Hydra") # change this to your own path
import hydra

# All things astropy
from astropy import units
from astropy.coordinates import EarthLocation, SkyCoord
from astropy.coordinates.builtin_frames import AltAz, ICRS
from astropy.time import Time

# Pandas, dataframe
import pandas as pd

# Time
from numba import jit
import time

# Parallel computing
from joblib import Parallel, delayed

# Functions

## Functions from `wigner_rotations.ipynb`

In [2]:
# def get_zenith_coord(nside, location, obstime):
#     """
#     Uses location and obstime to calculate the RA/Dec coordinates and pixel index of zenith above the given location.
    
#     ** Inputs:
    
#     location: given as an astropy `EarthLocation` object
#     obstime: given as an astropy `Time` object
#     nside: Resolution
    
#     ** Outputs:
    
#     ra_zenith: Right ascention of zenith at given location and obstime in radian
#     dec_zenith: Declination of zenith at given location and obstime in radian
#     lst: Local sidereal time corresponding to the given opstime and longitude in radian
#     pix_zenith: The index of the pixel at zenith given the resolution (nside)
    
#     """
    
#     # Define AltAz frame at time 0
#     frame_altaz0 = AltAz(obstime=obstime, location=location)
    
#     # # Define a coordinate that is at the zenith at time 0
#     zenith_at_loc_altaz = SkyCoord(alt=90.*units.degree, az=0.*units.degree, frame=frame_altaz0)
#     zenith_at_loc_radec = zenith_at_loc_altaz.copy().transform_to('icrs')

#     # Get zenith coordinates
#     ra_zenith = zenith_at_loc_radec.ra
#     dec_zenith = zenith_at_loc_radec.dec
    
#     # get local sidereal time of obstime0 at this longitude, in radians
#     lst = obstime.sidereal_time(kind='mean', longitude=location.lon)
    
#     # Get the zenith pixel number
#     pix_zenith = hp.ang2pix(nside, ra_zenith.deg, dec_zenith.deg, lonlat=True)
    
#     return ra_zenith.rad, dec_zenith.rad, lst.rad, pix_zenith

In [3]:
def get_index_dataframe(lmax):#, ell=0, m=0):
    '''
    Function to get the list of indices (i, ell, m) for ALL the modes in the `spherical` code
    
    ** Returns
    
    mode_indices: pandas dataframe with indices ordered as (i, ell, m) where i is the 'global' index of the alm-modes.
    
    '''
    # Number of modes 
    nmodes_wigner = lmax*(lmax+2)+1                                  # all modes
    nmodes_healpy = lmax * (2 * lmax + 1 - lmax) // 2 + lmax + 1     # only m>=0 modes

    # Create empty Modes object to get indices
    modes = spherical.Modes(np.zeros(shape=nmodes_wigner, dtype=np.complex128), spin_weight=0)

    #Get ell, m and global index i according to the `spherical` ordering
    indices = np.empty(shape=(nmodes_wigner,2))
    for ell_mode in range(0,lmax+1):
        for m_mode in range(-ell_mode,ell_mode+1):
            i = modes.index(ell=ell_mode,m=m_mode)
            indices[i] = [ell_mode, m_mode]

    indices_df =  pd.DataFrame(indices)
    indices_df.columns = ['ell', 'm']
    
    # List of wigner indices
    mode_indices = indices_df
   
    return mode_indices

In [4]:
def ordering(lmax, ell=0, m=0):
    '''
    Function to ensure the correct ordering of alm-modes when switching between `HEALpy` and `spherical`.
    Finds the indices to convert between the ordering of 'HEALpy' and 'spherical' using ALL modes
    
    ** Returns
    
    wigner2healpy: indices in a list to go from `spherical` ordering to `HEALpy`
    healpy2wigner: indices in a list to go from `HEALpy` ordering to `spherical`
    neg_modes_index: Index of where to cut off negative modes to get to `HEALpy`s alm-convention. Necessary for plotting.
    index_ell_mode: Gets `spherical` indices of the modes matching the input `ell` value Default ell=0 mode.
    index_m_mode: Gets `spherical` indices of the modes matching the input `m` value. Default m=0 mode.
    
    '''
    # Number of modes 
    nmodes_wigner = lmax*(lmax+2)+1                                  # all modes
    nmodes_healpy = lmax * (2 * lmax + 1 - lmax) // 2 + lmax + 1     # only m>=0 modes

    
    # List of wigner indices
    mode_indices = get_index_dataframe(lmax)#, ell, m)

    # Sort indices: spherical (wigner) -->  HEALpy
    df1 = mode_indices.sort_values(['m', 'ell'])
    wigner2healpy = df1.index.to_list()
    
    # Sort indices: HEALpy --> spherical (wigner)
    reset = df1.reset_index()  # resets to HEALpy as standard
    df2 = reset.sort_values(['ell', 'm'])
    healpy2wigner = df2.index.to_list()

    # Get ell, m indicies (ONLY FOR WIGNER)
    index_ell_mode = df1.loc[df1.ell == ell].index.tolist()
    index_m_mode = df1.loc[df1.m == m].index.tolist()
    
    # Get cutoff index for removing negative modes
    neg_modes_index = nmodes_wigner-nmodes_healpy
    
    return wigner2healpy, healpy2wigner, neg_modes_index, index_ell_mode, index_m_mode

In [144]:
def WignerDsize(ell_min, mp_max, ell_max=-1): # function taken from the `spherical` utilities, should probably replace with link?
    if ell_max < 0:
        ell_max = mp_max
    if mp_max >= ell_max:
        return (
            ell_max * (ell_max * (4 * ell_max + 12) + 11)
            + ell_min * (1 - 4 * ell_min**2)
            + 3
        ) // 3
    if mp_max > ell_min:
        return (
            3 * ell_max * (ell_max + 2)
            + ell_min * (1 - 4 * ell_min**2)
            + mp_max * (
                3 * ell_max * (2 * ell_max + 4)
                + mp_max * (-2 * mp_max - 3) + 5
            )
            + 3
        ) // 3
    else:
        return (ell_max * (ell_max + 2) - ell_min**2) * (1 + 2 * mp_max) + 2 * mp_max + 1

In [145]:
def WignerDindex(ell, mp, m, ell_min=0, mp_max=-1): # from the spherical utilities, should probably replace with link?
    """
    Compute index into Wigner D matrix
    
    Parameters
    ----------
    ell : int
        Integer satisfying ell_min <= ell <= ell_max
    mp : int
        Integer satisfying -min(ell_max, mp_max) <= mp <= min(ell_max, mp_max)
    m : int
        Integer satisfying -ell <= m <= ell
    ell_min : int, optional
        Integer satisfying 0 <= ell_min <= ell_max.  Defaults to 0.
    mp_max : int, optional
        Integer satisfying 0 <= mp_max.  Defaults to ell.
    
    Returns
    -------
    i : int
        Index into Wigner D matrix arranged as described below
    
    See Also
    --------
    WignerDsize : Total size of the D matrix
    WignerDrange : Array of (ℓ, m', m) indices corresponding to the 𝔇 matrix
    
    Notes
    -----
    This assumes that the Wigner D matrix is arranged as
        [
            𝔇(ℓ, mp, m)
            for ℓ in range(ell_min, ell_max+1)
            for mp in range(-min(ℓ, mp_max), min(ℓ, mp_max)+1)
            for m in range(-ℓ, ℓ+1)
        ]
    """
    if mp_max < 0:
        mp_max = ell
    i = (mp + min(mp_max, ell)) * (2 * ell + 1) + m + ell
    if ell > ell_min:
        i += WignerDsize(ell_min, mp_max, ell-1)
    return i

In [310]:
def D_matrix_1D(lmax, hour_angle, ra=0, dec=0):
    """
    Calculcates the 1D D-matrix for a specific rotation given by the hour angle
    
    Parameters
    ----------
    
    * lmax (int): 
            Sets the number of modes (used for creating the `Wigner` object)
    * ha (float): 
            The hour angle of the source. Assumes that it's given as a fraction of a 
            sidereal day, i.e. as a decimal number where 1 = one sidereal day.  
    * ra (float):
            The right ascension of the source in RADIANS. Only set this if you want an 
            offset from the centre. Default: ra = 0.
    * dec (float): 
            The declination of the source in RADIANS. Only set this if you want an 
            offset from the centre. Default: dec = 0.         
    
    Returns
    -------
    
    * wigner_D_matrix_1D (spherical wigner matrix object (1D array)):
            Rotational matrix to perform the hour_angle rotation.
    
    Notes
    ------

    Rotation Euler angles for `spherical`:
    
      * zyz-convention
      * Order of rotations: phi --> theta --> psi
      * phi and theta are swapped compared to astropy 
      * phi has the opposite sign compared to astropy
     
    phi: (radian) Corresponds to change in HA. Sign is automatically changed to fit with `spherical`
    theta: (radian) Corresponds to declination.
    psi: (radian) Is per default set to zero.
    """
       
    hour_angle_rad = np.radians(hour_angle*360)
              
    # Set euler angles with 'spherical' sign convention
    phi = ra+hour_angle_rad
    theta = dec
    psi = 0


    # Calculate rotation of a_lms
    wigner = spherical.Wigner(ell_max=lmax)
    rotate_to_coord = quaternionic.array.from_euler_angles(psi, theta, -phi)
    wigner_D_matrix_1D = wigner.D(rotate_to_coord)
        
    return wigner_D_matrix_1D

In [8]:
def D_matrix_per_ell_2D(D_matrix_as_1D, ell, lmax):
    """
    Unpacks the one-dimensional D-matrix into separate 2D-D-matrices 
    for the specific ell-mode (as the individual square D-matrices have 
    different sizes for each ell-mode)
    
    Parameters
    ----------
    
    * D_matrix_as_1D (spheicral D-matrix object (1D array)):
            Rotational matrix to perform the hour_angle rotation. One dimensional.
    * ell (int):
            Specific ell-mode to calculate the two-dimensional D-matrix for
    * lmax (int): 
            Sets the number of modes (used for creating the `Wigner` object)      
    
    Returns
    -------
    
    * D_matrix_as_2D (ndarray, 2D):
            Rotational matrix to perform the hour_angle rotation for the given 
            ell-mode
    
    
    """
    
    assert ell <= lmax, "ell has to be smaller or equal to lmax (ell <= lmax)"
    
    m_range = np.arange(-ell,ell+1)
    
    # Make empty 2D D-matrix with correct shape given by m x mp
    D_matrix_as_2D = np.zeros(shape=(np.size(m_range),np.size(m_range)), dtype=complex)
        
    m_count = 0
    for m in m_range:
        
        mp_count=0

        for mp in np.arange(-ell, ell+1):
            # Get the correct value from the 1D D-matrix
            index = WignerDindex(ell, mp, m, ell_min=0, mp_max=lmax)
            value = D_matrix_as_1D[index]
            
            # Save it into the correct index in the 2D D-matrix
            D_matrix_as_2D[m_count, mp_count] = value
    
            mp_count += 1

        m_count += 1
    
    return D_matrix_as_2D

In [9]:
# def rotate_modes_D_matrix(D_matrix_1D, modes, lmax):
#     '''
#     Calculate the separate D-matrices per ell-value, apply the rotations, combine into one modes-array
    
#     Parameters
#     ----------
    
#     * D_matrix_1D (spheicral D-matrix object (1D array)):
#             Rotational matrix to perform the hour_angle rotation. One dimensional.
#     * modes (`spherical` modes object (complex)):
#             Array of complex alms including both (+/-)m-modes.
#             The array is ordered to fits with the `spherical code`.
#     * lmax (int): 
#             Sets the number of modes (used for creating the `Wigner` object)      
    
#     Returns
#     -------
    
#     * rotated_modes (`spherical` modes object (complex)):
#             Array of *rotated* complex alms including both (+/-)m-modes.
#             The array is ordered to fits with the `spherical code`.
       
#     '''
#     # get pandas dataframe with indices ordered as (i, ell, m)
#     my_wigner_df = get_index_dataframe(lmax)
#     all_ell_list = my_wigner_df.copy().to_numpy()[:,0]
    
    
#     D_rot_modes = np.zeros(shape=0, dtype=complex)

#     for ell in np.arange(0,lmax+1):
#         ell_value = ell
#         indices = np.where(all_ell_list == ell_value)
#         modes_per_ell = np.dot(D_matrix_per_ell_2D(D_matrix_1D, ell_value, lmax), modes[indices])
#         rotated_modes = np.concatenate((rotated_modes, modes_per_ell), axis=0)
        
#     return rotated_modes

In [10]:
def combined_D_matrix(D_matrix_1D, lmax):
    '''
    Calculate the separate 2D D-matrices per ell-value, combined into one block diagonal matrix.
    The rotated modes can then be calculated by a dot product: 
                    
                    rotated_modes = np.dot(combined_D_matrix(D_matrix, lmax), modes)
     
    Parameters
    ----------
   
    * D_matrix_1D (spheicral D-matrix object (1D array)):
             Rotational matrix to perform the hour_angle rotation. One dimensional.

    * lmax (int): 
             Sets the number of modes (used for creating the `Wigner` object)      
    
    Returns
    -------
    
    * combined_D_matrix (ndarray):
            Rotational matrix to perform the rotation via dot product


    '''
   
    combined_D = np.zeros(shape=0, dtype=complex)

    for ell in np.arange(0,lmax+1):
        ell_value = ell
        
        D_matrix_2D = D_matrix_per_ell_2D(D_matrix_1D, ell_value, lmax)
        
        if ell_value == 0:
            combined_D_matrix = D_matrix_2D
        else:
            combined_D_matrix = sp.linalg.block_diag(combined_D_matrix, D_matrix_2D)
        
    return combined_D_matrix

# np.shape(combined_D_matrix(D_matrix, lmax))

In [11]:
# def spherical_rotate_modes(modes, lmax, theta, phi, psi=0):
#     """ Performs the actual ROTATIONS of the modes directly using the `spherical`
#     code without calculating the wigner 𝔇-matrix.
    
#     Parameters
#     ----------

#     modes (`spherical` modes object):
#         Wigner/`spherical` modes to rotate. Do not give it HEALpy alms
#     lmax (int): 
#         Sets the number of modes (used for creating the `Wigner` object)

#     phi: (radian) Corresponds to RA. Sign is automatically changed to fit with `spherical`
#     theta: (radian) Corresponds to declination.
#     psi: (radian) Is per default set to zero, but can be set if wanted.
    
    
#     Returns
#     -------
    
#     modes_rot (`spherical` modes object)
#         The new/rotated modes given the input angles
    
#     Notes
#     -----
#     This assumes rotation Euler angles for `spherical` given by:
    
#       * zyz-convention
#       * Order of rotations: phi --> theta --> psi
#       * phi and theta are swapped compared to astropy 
#       * phi has the opposite sign compared to astropy

#       ==> phi is equivalent to negative longitude/ RA
#       ==> theta is latitude / dec>
#       ==> psi should be set to zero
      
#     """
#     # FixMe: Make an assertion that the angles are actaully given in radian

#     # Calculate rotation of a_lms
#     wigner = spherical.Wigner(ell_max=lmax)
#     rotate_to_coord = quaternionic.array.from_euler_angles(psi, theta, -phi)
#     rotated_modes = wigner.rotate(modes, rotate_to_coord)
        
#     return rotated_modes

## New rotation functions

In [135]:
def apply_D_matrix(block_diag_D_matrix, modes, lmax):
    '''
    Perform the rotation by taking a dot product with the wigner modes and
    re-defining them as a wigner object.
    
    Parameters
    ----------
    
    * bloack_diag_D_matrix(array):
            Rotational matrix to perform the hour_angle rotation. Can be calculated
            from the one-dimensional D-matrix by `combined_D_matrix(D_matrix_1D, lmax)`
    * modes (`spherical` modes object (complex)):
            Array of complex alms including both (+/-)m-modes.
            The array is ordered to fits with the `spherical code`.
    * lmax (int): 
            Sets the number of modes (used for creating the `Wigner` object)      
    
    Returns
    -------
    
    * rotated_modes (`spherical` modes object (complex)):
            Array of *rotated* complex alms including both (+/-)m-modes.
            The array is ordered to fits with the `spherical code`.
       
    '''
    
    apply_D_matrix = np.dot(block_diag_D_matrix, modes)
    rotated_modes = spherical.Modes(apply_D_matrix, spin_weight=0)

    return rotated_modes
    

## New functions for manipulation of modes

In [314]:
def create_alms(mode, lmax):
    """
    Creating a double array of alms with real values only. 
    The output array represents all positive (+m) modes including zero 
    and has double length, as real and imaginary values are split. 
    The first half is the real values. 
    
    All modes in the array is set to zero except for the mode set by 
    "mode". This should be interpreted as HEALpy ordering, meaning (m,l)
    
    Parameters
    ----------
    
    * mode (int):
            The mode index that you want to set to 1. 
    * lmax (int):
            Maximum ell-value. Is only used to determine the total
            number of modes
            
    
    Returns
    -------
    
    * alms (ndarray):
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zero 
            and has double length, as real and imaginary values are split. 
            The first half is the real values.
    
    Notes
    ------
    
    The number of modes is set by the HEALpy convention
    
        nmodes_healpy = lmax * (2 * lmax + 1 - lmax) // 2 + lmax + 1 
    
    and then multiplies by two to separate the real and imaginary part ( in
    order to have and effectively real array)
    
         nalms = 2 * nmodes_healpy
    
    """
    
    nmodes_healpy = lmax * (2 * lmax + 1 - lmax) // 2 + lmax + 1 
    nalms = 2 * nmodes_healpy
    alms = np.zeros(nalms, dtype=np.float64)
    alms[mode] = 1.    
    
    return alms

In [14]:
def retrieve_alms(mode, nalms):
    """
    Retrieving a double array of alms based on the total number of modes 
    present. The array has real values only. The output array represents 
    all positive (+m) modes including zero and has double length, as real
    and imaginary values are split. The first half is the real values. 
    
    All modes in the array is set to zero except for the mode set by 
    "mode". This should be interpreted as HEALpy ordering, meaning (m,l)
    
    Parameters
    ----------
    
    * mode (int):
            The mode index that you want to set to 1. 
    * nalms (int):
            The total number of modes for a _real_ array of alms 
            (i.e. where the first half represents real-values and
            the second half represents imaginary values (real, imag))
            
    
    Returns
    -------
    
    * alms (ndarray):
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zero 
            and has double length, as real and imaginary values are split. 
            The first half is the real values.
    
    """
    
    alms = np.zeros(nalms, dtype=np.float64)
    alms[mode] = 1    
    
    return alms

In [15]:
def split_complex_array(complex_array):
    """
    Takes a complex array and turns into a real array of double size 
    on axis=0, split as [real, imag].
    
         Parameters
    ----------
    * complex_array (ndarray (complex)):
            Numpy array containing complex values.
    
    Returns
    -------
    * alms (ndarray of floats)
           Real array of double size on axis 0. It is ordered with the real
           values first and then the complex values. 
    
    """
    
    real_array = np.concatenate((complex_array.real,complex_array.imag))

    return real_array

In [16]:
def alms2modes(alms, lmax):
    """
    Takes an array of (real) (l,+m)-alms where first half represent the real 
    values and the last half represents the imaginary values and 
    recombines it into a complex array including the (l, -m)-modes and 
    reorder to fit with the `spherical` (wigner) ordering.
    
     Parameters
    ----------
    
    * alms (ndarray of floats):
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zero 
            and has double length, as real and imaginary values are split. 
            The first half is the real values.
    * lmax (int):
            Maximum ell-value. Determines the number of alms.    
            
    
    Returns
    -------
    
    * wigner_modes (`spherical` modes object (complex)):
            Array of complex alms including both (+/-)m-modes.
            The array is ordered to fits with the `spherical code`.

    
    Notes
    ------
    
    When the negative m-modes are included, this rule is needed 
    to obtain a real field (for all ell):
    
        (a_l,+m).real = -(a_l,-m).real
        (a_l,+m).imag = +(a_l,-m).imag
      
    """

    healpy_modes = alms[:int(np.size(alms)/2)] + 1.j*alms[int(np.size(alms)/2):]
    n_negative_modes = int((lmax**2 + lmax)/2)    

    # Rearrange the negative modes into correct HEALpy ordering              ## THIS IS THE BOTTLENECK 
    temp = healpy_modes[-n_negative_modes:]
    neg_modes = np.empty(0, dtype=np.complex128)
    for i in np.arange(1,lmax+1):  
        keep = temp[-i:]
        neg_modes = np.append(neg_modes,keep)

        temp = temp[:-i]

    # Assuring a real field
    neg_modes = -neg_modes.real + 1.j*neg_modes.imag
    
    # Reordering and creating `spherical` modes object
    combined_modes = np.concatenate((neg_modes,healpy_modes))
    _, healpy2wigner, _, _, _ = ordering(lmax)
    wigner_modes = spherical.Modes(combined_modes[healpy2wigner], spin_weight=0)
   
    return wigner_modes

In [18]:
def modes2healpy(modes, lmax):
    """
    Takes a `spherical` Modes object of complex alms (including both
    positive and negative m-modes) and turns it into a complex array of 
    alms without the negative m-modes.
    The final array is ordered as in HEALpy.
    
     Parameters
    ----------
    * wigner_modes (`spherical` modes object (complex)):
            Array of complex alms including both (+/-)m-modes.
            The array is ordered to fits with the `spherical code`.
    
    * lmax (int):
            Maximum ell-value. Determines the number of alms.    
            
    
    Returns
    -------
    * healpy_modes (ndarray (complex))
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zeroth modes.
    """
    
    wigner2healpy, _, neg_modes_index, _, _ = ordering(lmax)
    healpy_modes = modes.ndarray[wigner2healpy][neg_modes_index:]
                                           
    return healpy_modes

In [19]:
def healpy2alms(healpy_modes):
    """
    Takes a complex array of alms (positive modes only) and turns into
    a real array of double size split as [real, imag].
      
     Parameters
    ----------
    * healpy_modes (ndarray (complex)):
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zeroth modes.
    
    Returns
    -------
    * alms (ndarray (floats))
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zero 
            and has double length, as real and imaginary values are split. 
            The first half is the real values.
    """
    
    #alms = np.concatenate((healpy_modes.real,healpy_modes.imag))
    alms = split_complex_array(healpy_modes)
        
    return alms    

In [20]:
def alms2healpy(alms):
    """
    Takes a real array of 2*(number of modes) size split as [real, imag]
    and turns it into a complex array of alms (positive modes only) ordered
    as in HEALpy.
      
     Parameters
    ----------
    * alms (ndarray (floats))
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zero 
            and has double length, as real and imaginary values are split. 
            The first half is the real values.

    
    Returns
    -------
    * healpy_modes (ndarray (complex)):
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zeroth modes.
            
    """
    healpy_modes = alms[:int(np.size(alms)/2)] + 1.j*alms[int(np.size(alms)/2):]
    
    return healpy_modes

## Functions from `spherical_harmonic_visibility_simulator`

In [21]:
# @jit
def delay(alt, az, bl):
    """
    Calculate the delay, tau, in ns between a baseline vector [x, y, z] 
    in metres, where x points East, and an Alt/Az coordinate (where 
    Azimuth = 0 in the East). The Alt/Az coords should be in radians.
    """
    C = 299792458. # m/s
    
    # Calculate zenith angle
    alt = np.atleast_1d(alt)
    az = np.atleast_1d(az)
    za = 0.5*np.pi - alt
    
    # Source vector
    src_vec = np.array([np.sin(za)*np.cos(az), 
                        np.sin(za)*np.sin(az),
                        np.cos(za)]).T
    
    # Return dot product
    bl = np.atleast_2d(bl).T
    return np.dot(src_vec, bl) / C * 1e9 # ns


# @jit
def fringe(alt, az, bl, freq=100.):
    """
    Calculate the fringe pattern for a point source with position `alt`/`az` 
    (in rad), baseline vector `bl` ([x,y,z] in metres), and at frequency 
    `freq` (in MHz).
    """
    tau = delay(alt, az, bl) # in ns
    return np.exp(1.j * 2.*np.pi * tau * freq*1e-3)


# @jit
def horizon(alt, az):
    m = np.ones(alt.size)
    m[alt < 0.] = 0.
    return m


# @jit
def gaussian_beam(alt, az, freq, diameter=14.):
    C = 299792458. # m/s
    width = C / (100.*1e6) / diameter # rad
    return np.exp(-0.5 * (alt - 0.5*np.pi)**2. / width**2.)

## Basic visibility response function

In [17]:
def modes2alms(modes, lmax):
    """
    Takes a `spherical` Modes object of complex alms (including both
    positive and negative m-modes) and turns it into a real array of 
    alms by removing the negative m-modes and splitting into real and
    imaginary parts. The final array is ordered as in HEALpy, but 
    double in size.
    
     Parameters
    ----------
    * wigner_modes (`spherical` modes object (complex))
            Array of complex alms including both (+/-)m-modes.
            The array is ordered to fits with the `spherical code`.
    
    * lmax (int):
            Maximum ell-value. Determines the number of alms.    
            
    
    Returns
    -------
    * alms (ndarray of floats)
            Array of zeros except for the specified mode. 
            The array represents all positive (+m) modes including zero 
            and has double length, as real and imaginary values are split. 
            The first half is the real values.
    """
    
    wigner2healpy, _, neg_modes_index, _, _ = ordering(lmax)
    healpy_modes = modes.ndarray[wigner2healpy][neg_modes_index:]
    alms = np.concatenate((healpy_modes.real, healpy_modes.imag))
    
    return alms

In [22]:
# FIXME: healpy function nside2npix seems to cause issues unless forceobj=True
#@jit(parallel=True, forceobj=True)
def vis_sh_response_multi_rotations(bl, freqs, lmax, hour_angles=0, nside=64):
    """
    Calculate the response of a baseline to a spherical harmonic 
    mode with unit amplitude (a_lm = 1) on the sky with a flat 
    frequency spectrum.
    
    Parameters:
        bl (array_like):
            Baseline [x,y,z] position in metres.

        freqs (array_like):
            Array of frequency values in MHz.

        lmax (int):
            Maximum ell value of spherical harmonic 
            modes to calculate.
        
        nside (int):
            Healpix nside to use for the calculation (longer 
            baselines should use higher nside).
        
        hour_angle (float):
            Hour angle given as fraction in decimal of a sidereal day. 
    
    Returns:
        response (array_like):
            Visibility V_ij for each (l,m) mode, with shape 
            (N_alm, N_freqs).
    """
    # Set the correct size of modes (double to account for real and imag-part)
    nmodes_healpy = lmax * (2 * lmax + 1 - lmax) // 2 + lmax + 1 
    nalms = 2 * nmodes_healpy
    
    # Get alt/az values for each pixel
    npix = hp.nside2npix(nside)
    ii = np.arange(npix)
    az, alt = hp.pix2ang(nside, ii, lonlat=True)
    alt = np.deg2rad(alt.flatten()) # radians
    az = np.deg2rad(az.flatten()) # radians
    
    # Empty visibility response array
    vis = np.zeros((hour_angles.size, nalms, freqs.size), dtype=np.complex128)
    
    # Horizon mask
    mask_horizon = horizon(alt, az)
    
    for k in range(hour_angles.size):
        # Calculate D-matrix
        D_matrix = combined_D_matrix(D_matrix_1D(lmax, hour_angles[k]), lmax)

        # Loop over frequencies
        for i in range(freqs.size):

            # Calculate fringe pattern times Gaussian primary beam squared times horizon mask
            fringe_times_beamsq = fringe(alt, az, bl, freqs[i]).flatten() \
                                  * gaussian_beam(alt, az, freqs[i])**2. \
                                  * mask_horizon

            # Loop over (l,m) modes
            for j in range(nalms):

                # Switch on one a_lm mode at a time and generate map
                alms = create_alms(mode=j,lmax=lmax)
                modes = alms2modes(alms,lmax)
                rotated_modes = rotate_D_matrix(D_matrix, modes, lmax)
                healpy_modes = modes2healpy(rotated_modes, lmax)

                skymap = hp.sphtfunc.alm2map(healpy_modes, nside=nside)

                # Calculate integral (FIXME: unnormalised!)
                vis[k,j,i] = np.sum(skymap * fringe_times_beamsq)
                
        
    
    return vis

In [23]:
#@jit(parallel=True, forceobj=True)
def vis_sh_response_rot(bl, freqs, lmax, hour_angle=0, nside=64):
    """
    Calculate the response of a baseline to a spherical harmonic 
    mode with unit amplitude (a_lm = 1) on the sky with a flat 
    frequency spectrum.
    
    Parameters:
        bl (array_like):
            Baseline [x,y,z] position in metres.

        freqs (array_like):
            Array of frequency values in MHz.

        lmax (int):
            Maximum ell value of spherical harmonic 
            modes to calculate.
        
        nside (int):
            Healpix nside to use for the calculation (longer 
            baselines should use higher nside).
        
        hour_angle (float):
            Hour angle given as fraction in decimal of a sidereal day. Default: no rotation.
    
    Returns:
        response (array_like):
            Visibility V_ij for each real- and imaginary part of the (l,m) modes, with shape 
            (2xN_alm, N_freqs).
    """
    # Set the correct size of modes (double to account for real and imag-part)
    nmodes_healpy = lmax * (2 * lmax + 1 - lmax) // 2 + lmax + 1 
    nalms = 2 * nmodes_healpy
    
    # Get alt/az values for each pixel
    npix = hp.nside2npix(nside)
    ii = np.arange(npix)
    az, alt = hp.pix2ang(nside, ii, lonlat=True)
    alt = np.deg2rad(alt.flatten()) # radians
    az = np.deg2rad(az.flatten()) # radians
    
    # Empty visibility response array
    vis = np.zeros((nalms, freqs.size), dtype=np.complex128)
    
    # Horizon mask
    mask_horizon = horizon(alt, az)
    

    # Calculate D-matrix
    D_matrix = combined_D_matrix(D_matrix_1D(lmax, hour_angle), lmax)

    # Loop over frequencies
    for i in range(freqs.size):

        # Calculate fringe pattern times Gaussian primary beam squared times horizon mask
        fringe_times_beamsq = fringe(alt, az, bl, freqs[i]).flatten() \
                              * gaussian_beam(alt, az, freqs[i])**2. \
                              * mask_horizon

        # Loop over (l,m) modes
        for j in range(nalms):

            # Switch on one a_lm mode at a time and generate map
            alms = create_alms(mode=j,lmax=lmax)
            modes = alms2modes(alms,lmax)
            rotated_modes = rotate_D_matrix(D_matrix, modes, lmax)
            healpy_modes = modes2healpy(rotated_modes, lmax)

            skymap = hp.sphtfunc.alm2map(healpy_modes, nside=nside)

            # Calculate integral (FIXME: unnormalised!)
            vis[j,i] = np.sum(skymap * fringe_times_beamsq)
                
        
    
    return vis

In [24]:
#@jit(parallel=True, forceobj=True)
def vis_sh_response_basic(bl, freqs, lmax, nside=64):
    """
    Calculate the response of a baseline to a spherical harmonic 
    mode with unit amplitude (a_lm = 1) on the sky with a flat 
    frequency spectrum.
    
    Parameters:
        bl (array_like):
            Baseline [x,y,z] position in metres.

        freqs (array_like):
            Array of frequency values in MHz.

        lmax (int):
            Maximum ell value of spherical harmonic 
            modes to calculate.
        
        nside (int):
            Healpix nside to use for the calculation (longer 
            baselines should use higher nside).
           
    Returns:
        response (array_like):
            Visibility V_ij for each real- and imaginary part of the (l,m) modes, with shape 
            (2xN_alm, N_freqs).
    """
    # Set the correct size of modes (double to account for real and imag-part)
    nmodes_healpy = lmax * (2 * lmax + 1 - lmax) // 2 + lmax + 1 
    nalms = 2 * nmodes_healpy
    
    # Get alt/az values for each pixel
    npix = hp.nside2npix(nside)
    ii = np.arange(npix)
    az, alt = hp.pix2ang(nside, ii, lonlat=True)
    alt = np.deg2rad(alt.flatten()) # radians
    az = np.deg2rad(az.flatten()) # radians
    
    # Empty visibility response array
    vis = np.zeros((nalms, freqs.size), dtype=np.complex128)
    
    # Horizon mask
    mask_horizon = horizon(alt, az)

    # Loop over frequencies
    for i in range(freqs.size):

        # Calculate fringe pattern times Gaussian primary beam squared times horizon mask
        fringe_times_beamsq = fringe(alt, az, bl, freqs[i]).flatten() \
                              * gaussian_beam(alt, az, freqs[i])**2. \
                              * mask_horizon

        # Loop over (l,m) modes
        for j in range(nalms):

            # Switch on one a_lm mode at a time and generate map
            alms = create_alms(mode=j,lmax=lmax)
            healpy_modes = alms2healpy(alms)

            skymap = hp.sphtfunc.alm2map(healpy_modes, nside=nside)

            # Calculate integral (FIXME: unnormalised!)
            vis[j,i] = np.sum(skymap * fringe_times_beamsq)
                
        
    
    return vis

## Precompute and apply operator

In [210]:
def precompute_op(freqs, lst_ref, beams, ant_pos, lmax, nside):
    """
    Precompute the real and imaginary blocks of the visibility response 
    operator. This should only be done once and then "apply_operator()"
    is used to get the actual visibilities.
    
    Parameters
    ----------
   
    * freqs (array_like):
            Frequencies, in MHz.
    
    * lst_ref (float or int):
            Reference LST of the observation, in radians.
    
    * beams (list of pyuvbeam):
            List of pyuveam objects, one for each antenna
            
    * ant_pos (dict):
            Dictionary of antenna positions, [x, y, z], in m. The keys should
            be the numerical antenna IDs.    
            
    * lmax (int):
            Maximum ell value. Determines the number of modes used.
             
    * nside (int):
            Healpix nside to use for the calculation (longer baselines should 
            use higher nside).
    
    Returns
    -------
    
    * vis_response (array_like):
            Visibility response (δV_ij) for each (l,m) mode, frequency and 
            baseline for a given reference LST. Shape (N_bl, N_freqs, N_alms).
    
    """
       
    ell, m, vis_alm = hydra.vis_simulator.simulate_vis_per_alm(lmax=lmax, 
                                                           nside=nside, 
                                                           ants=ant_pos, 
                                                           freqs=freqs, 
                                                           lsts=lst_ref, 
                                                           beams=beams)
    ants = list(ant_pos.keys())
    antpairs = []
    for i in ants:
        for j in ants:
            if j >= i:
                antpairs.append((ants[i],ants[j]))
    
    vis_response = np.zeros((len(antpairs), len(freqs), 2*len(ell)), dtype=np.complex128)

    for i, bl in enumerate(antpairs):
        idx1 = ants.index(bl[0])
        idx2 = ants.index(bl[1])
        vis_response[i, :, :] = vis_alm[:, 0, idx1, idx2, :]
        
    return vis_response

In [226]:
def apply_operator(alms, vis_response, times, lmax):
    """
    Applys visibility reponse operator to a set of alms for given times/rotations
    of hour angles.
    
    Parameters
    ----------
    * alms (ndarray of floats):
            Array of spherical harmonic alm-modes. The array represents all positive 
            (+m) modes including zero and has double length, as real and imaginary 
            values are split. The first half is the real values.
            
    * vis_response (array_like):
            Visibility response (δV_ij) for each (l,m) mode, frequency and 
            baseline for a given reference LST. Shape (N_bl, N_freqs, N_alms).
            
    * times (array):
            Array of observation times given as a fractions of a sidereal day, where 
            1 is a full sidereal day. 
            
    * lmax (int):
            Maximum ell value. Determines the number of modes used.
    
    Returns
    -------
    
    * Visibility (array_like):
            Visibility Vij with shape (N_bl, N_times, N_freqs) 
    
    """
    
    modes = alms2modes(alms,lmax)
    
    Nbl = vis_response.shape[0]
    Nfreqs = vis_response.shape[1]
    
    visibility = np.zeros((Nbl, len(times), Nfreqs), dtype=np.complex128)
    
    for i in range(times.size):
        D_matrix = combined_D_matrix(D_matrix_1D(lmax, times[i]), lmax)
        rotated_modes = apply_D_matrix(D_matrix, modes, lmax)

        visibility[:,i,:] = vis_response @ modes2alms(rotated_modes,lmax)
    
    return visibility


# [EXAMPLE SCRIPT] for the full calculation

### Choosing resolution: lmax and nside

In [315]:
lmax = 15
nside = 64

### Setting antennas, beams, frequencies, reference lst and rotations/times

In [316]:
ant_pos = {0: (0., 0., 0.), 
           1: (14., 0., 0.)}
beams = [pyuvsim.AnalyticBeam('airy', diameter=8.) for ant in ants]
freqs=np.linspace(100e6, 200e6, 20)

# One reference lst (there's no thought put into this choice) 
lst_ref = 1.

# Time of rotation given as fraction of a sidereal day
hour_angles = np.linspace(0, 1, 50)# * units.hour

### [!] alms to test on, these are the variable to sample in the sampler though, so this should not be included there

In [317]:
nmodes_healpy = lmax * (2 * lmax + 1 - lmax) // 2 + lmax + 1  
nalms = 2*nmodes_healpy
mock_alms = np.random.rand(nalms)*10

### Run `precompute_op()` and `apply_operator()`

In [318]:
t0 = time.time()
vis_response = precompute_op(freqs, lst_ref, beams, ant_pos, lmax, nside)
visibility = apply_operator(mock_alms, vis_response, hour_angles, lmax)
print("Run took %5.3f sec" % (time.time() - t0))

Run took 7.986 sec


# [✓] Test that it retrieves the same thing within a margin of error of 1e-06

### [✓] Are modes rotated with hour angle = 0 the same as the original non-rotated modes?

In [355]:
err = 1e-13

In [356]:
D_matrix = combined_D_matrix(D_matrix_1D(lmax, hour_angles[0]), lmax)
rotated_modes = apply_D_matrix(D_matrix, alms2modes(mock_alms,lmax), lmax)

In [357]:
print("Are the 0-hr rotated and original alms the same? ",np.all((rotated_modes.ndarray - alms2modes(mock_alms,lmax).ndarray) <= (0 + err)) == True)

Are the 0-hr rotated and original alms the same?  True


### [✓] Is the visibility of rotating of an hour angle of 0, the same as applying the vis_response directly to the alms
### [✓] Also a check that it matches zeroth rotation of the big visibility array

In [358]:
visibility0 = apply_operator(mock_alms, vis_response, np.array([0]), lmax)
visibility00 = vis_response @ mock_alms

print("Are the 0hr rotated visibilities the same as not rotating?")
print(np.all((visibility0[:,0,:] - visibility[:,0,:])  <= (0 + err)) == True)
print(np.all((visibility0[:,0,:] - visibility00)  <= (0 + err)) == True)
print(np.all((visibility[:,0,:] - visibility00)  <= (0 + err)) == True)

Are the 0hr rotated visibilities the same as not rotating?
True
True
True


# "Note book/work book":

## Using the basic visibility response function

In [25]:
# Reference observation time
obstime0 = Time('2022-06-27T01:02:30.11', format='isot', scale='utc') 

# Second observation time 
obstime1 = Time('2022-06-27T03:02:30.11', format='isot', scale='utc') 

# Time of rotation given as fraction of a sidereal day
hour_angles = np.linspace(0, 1, 24)# * units.hour

lmax = 15
nside = 64

bl = [14.,0.,0.]
freqs=np.linspace(100., 200., 20)

## Precomputing the basic response function

In [26]:
t0 = time.time()
vis_response = vis_sh_response_basic(bl, freqs, lmax, nside=64)
time_response = time.time() - t0
print("Run took %5.3f sec" % (time_response))

Run took 4.103 sec


## Precomputing the D-matrices

In [27]:
nmodes_wigner = lmax*(lmax+2)+1

In [28]:
t0 = time.time()
D_matrices = np.zeros((hour_angles.size,nmodes_wigner, nmodes_wigner), dtype=np.complex128)

for i in range(hour_angles.size):
    # Calculate D-matrix
    D_matrices[i,:,:] = combined_D_matrix(D_matrix_1D(lmax, hour_angles[i]), lmax)
    
time_d_matrix = time.time() - t0
print("Run took %5.3f sec" % (time_d_matrix))

Run took 0.300 sec


#### Check that it stores it correctly:

In [29]:
D_matrix_0 = combined_D_matrix(D_matrix_1D(lmax, hour_angles[0]), lmax)

In [30]:
D_matrices[0,:,:] == D_matrix_0

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

## Rotating the alms

First creating some mock alms to check shapes work. Note that these are 0 everywhere and 1 at the given mode, so not really representative of anything but the size

In [31]:
mock_alms = create_alms(mode=42,lmax=lmax)

modes = alms2modes(mock_alms,lmax)

In [32]:
# t0 = time.time()
# for i in range(hour_angles.size):
#     if i == 0:      
#         rotated_modes = rotate_D_matrix(D_matrices[i,:,:], modes, lmax)
#         healpy_modes = modes2healpy(rotated_modes, lmax)
#         skymap0 = hp.sphtfunc.alm2map(healpy_modes, nside=nside)
#         skymaps = np.zeros((hour_angles.size,skymap0.size))
#         skymaps[i] = skymap0
#     else:
#         rotated_modes = rotate_D_matrix(D_matrices[i,:,:], modes, lmax)
#         healpy_modes = modes2healpy(rotated_modes, lmax)
#         skymaps[i] = hp.sphtfunc.alm2map(healpy_modes, nside=nside)

# time_rotation = time.time() - t0
# print("Run took %5.3f sec" % (time_rotation))

In [136]:
t0 = time.time()
rotated_alms = np.zeros((hour_angles.size, mock_alms.size))
for i in range(hour_angles.size):    
    rotated_modes = apply_D_matrix(D_matrices[i,:,:], modes, lmax)
    rotated_alms[i,:] = modes2alms(rotated_modes,lmax)

time_rotation = time.time() - t0
print("Run took %5.3f sec" % (time_rotation))

Run took 0.120 sec


## Combining the response and rotations

### Correcting the shapes so that they can be multiplied

In [34]:
print(f"visibility response shape: (nalms, freq) = {vis_response.shape}")
# print(f"rotated maps shape: (hour_angles, pixel) = {skymaps.shape}")
print(f"rotated healpy_modes has shape (hour_angle, nalms) ={rotated_alms.shape}")

visibility response shape: (nalms, freq) = (272, 20)
rotated healpy_modes has shape (hour_angle, nalms) =(24, 272)


In [35]:
vis_response_extra_axis = vis_response[np.newaxis,:, :]
rotated_alms_extra_axis = rotated_alms[:,:, np.newaxis]

visibility = vis_response_extra_axis * rotated_alms_extra_axis

In [36]:
visibility.shape

(24, 272, 20)

In [37]:
summed_vis = np.sum(visibility, axis=1)
summed_vis.shape

(24, 20)

## Total time of the functions in the run

In [38]:
total_time = time_response + time_d_matrix + time_rotation
print("Total run took %5.3f sec" % (total_time))

Total run took 4.500 sec


## Using the Hydra visibility response function

In [205]:
# Antenna array
ant_pos = {0: (0., 0., 0.),
        1: (14., 0., 0.)}

ants = list(ant_pos.keys())

# Beams
beams = [pyuvsim.AnalyticBeam('airy', diameter=8.) for ant in ants]

# Frequency
# freqs=[100e6,]
freqs=np.linspace(100., 200., 20)

# One reference lst (there's no thought put into this choice) 
lst_ref = 1.

# Time of rotation given as fraction of a sidereal day
hour_angles = np.linspace(0, 1, 50)# * units.hour

### Calculating visibilities pr alm-mode

In [168]:
t0 = time.time()
ell, m, vis_alm = hydra.vis_simulator.simulate_vis_per_alm(lmax=lmax, 
                                                           nside=nside, 
                                                           ants=ant_pos, 
                                                           freqs=freqs, 
                                                           lsts=lst_ref, 
                                                           beams=beams)
print("Run took %6.3f sec" % (time.time() - t0))

Run took  6.386 sec


True

In [169]:
#Complex, shape (N_freqs, N_lsts, N_ants, N_ants, N_alms)
vis_alm.shape

(20, 1, 2, 2, 272)

### Collapsing antenna dimensions into one baseline dimension and removing time dim from vis reponse

In [170]:
ants = list(ant_pos.keys())
antpairs = []
for i in ants:
    for j in ants:
        if j >= i:
            antpairs.append((ants[i],ants[j]))

print(antpairs)

Nants = len(ants)
Nbl = len(antpairs)
Nalms = 2*len(ell)   ## FIND WHERE YOU DID THIS PREVIOUSLY 
Nfreqs = len(freqs)

[(0, 0), (0, 1), (1, 1)]


In [181]:
"""
    ant_pos (dict):
    Dictionary of antenna positions, [x, y, z], in m. The keys should
    be the numerical antenna IDs.
    antpairs (list of tuple):
    List of tuples containing pairs of antenna IDs, one for each
    baseline.
"""
vis_response = np.zeros((Nbl, Nfreqs, Nalms), dtype=np.complex128)


ants = list(ant_pos.keys())
for i, bl in enumerate(antpairs):
    idx1 = ants.index(bl[0])
    idx2 = ants.index(bl[1])
    vis_response[i, :, :] = vis_alm[:, 0, idx1, idx2, :]

### Looping over hour_angles to calculate the total visibilites

First creating some mock alms to check shapes work. Note that these are 0 everywhere and 1 at the given mode, so not really representative of anything but the size

In [113]:
mock_alms = create_alms(mode=42,lmax=lmax)

modes = alms2modes(mock_alms,lmax)

In [137]:
t0 = time.time()
visibility = np.zeros((Nbl, hour_angles.size, Nfreqs),dtype=np.complex128)

for i in range(hour_angles.size):
    D_matrix = combined_D_matrix(D_matrix_1D(lmax, hour_angles[i]), lmax)
    rotated_modes = apply_D_matrix(D_matrix, modes, lmax)
    
    visibility[:,i,:] = vis_response[:,:,:] @ modes2alms(rotated_modes,lmax)
    
time_operator = time.time() - t0
print("Run took %5.3f sec" % (time_operator))

Run took 0.611 sec


In [134]:
print(visibility.shape)

(3, 24, 1)


In [140]:
vis_response.shape[2]

272

In [189]:
t0 = time.time()
vis_response = precompute_op(freqs, lst_ref, beams, ant_pos, lmax, nside)
visibility = apply_operator(mock_alms, vis_response, hour_angles, lmax)
print("Run took %5.3f sec" % (time.time() - t0))

Run took 8.179 sec


453

## [Not correct] First try at combining the precomputed D-matrix/alms and the visibility

First there should be a correction for shape and then .. multiplying them?

In [42]:
#Complex, shape (N_freqs, N_lsts, N_ants, N_ants, N_alms)
vis_alm.shape

(1, 1, 2, 2, 272)

In [43]:
rotated_alms.shape

(24, 272)

In [44]:
rotated_alms_modified = rotated_alms[:, np.newaxis, np.newaxis, np.newaxis, np.newaxis, :]
rotated_alms_modified.shape

(24, 1, 1, 1, 1, 272)

In [45]:
vis_alms_modified = vis_alm[np.newaxis, :, :, :, :, :]
vis_alms_modified.shape

(1, 1, 1, 2, 2, 272)

In [46]:
visibilities = vis_alms_modified*rotated_alms_modified
visibilities.shape

(24, 1, 1, 2, 2, 272)

NOTE! I need to sum over (l,m), so it should collapse the last dimension and leave the rest. Right?

In [47]:
# Now it has shape (N_rotatations, N_freqs, N_lsts, N_ants, N_ants)
summed_vis = np.sum(visibilities, axis=5)
summed_vis.shape

(24, 1, 1, 2, 2)

In [48]:
np.dot(vis_alms_modified,)

array([[[[[ 0.00000000e+00+0.00000000e+00j,
           -1.69406589e-21+0.00000000e+00j],
          [ 0.00000000e+00+0.00000000e+00j,
            0.00000000e+00+0.00000000e+00j]]]],



       [[[[ 0.00000000e+00+0.00000000e+00j,
            7.53080850e-22+8.80209449e-22j],
          [ 0.00000000e+00+0.00000000e+00j,
           -1.40833512e-20+0.00000000e+00j]]]],



       [[[[ 0.00000000e+00+0.00000000e+00j,
            2.98095968e-21+1.50413607e-21j],
          [ 0.00000000e+00+0.00000000e+00j,
           -2.40661771e-20+0.00000000e+00j]]]],



       [[[[ 0.00000000e+00+0.00000000e+00j,
            4.34089873e-21+1.69011664e-21j],
          [ 0.00000000e+00+0.00000000e+00j,
           -2.70418662e-20+0.00000000e+00j]]]],



       [[[[ 0.00000000e+00+0.00000000e+00j,
            4.43693655e-21+1.38400083e-21j],
          [ 0.00000000e+00+0.00000000e+00j,
           -2.21440133e-20+0.00000000e+00j]]]],



       [[[[ 0.00000000e+00+0.00000000e+00j,
            3.24111063e-21+6.7491769

## [Not correct] Creating the "opposite" function -- retrieving the alms

## Option 1, creating one long array of alms

In [44]:
nalms = vis[0,:,0].size
alms = []

for j in range(nalms):
    alm = retrieve_alms(mode=j, nalms=nalms)
    alms = np.append(alms, alm)
    
print(alms.shape)
print(nalms*nalms)

(73984,)
73984


## Option 2, creating a 2D alms array

In [66]:
nalms = vis[0,:,0].size
alms = np.zeros((nalms,nalms), dtype=np.float64)

for j in range(nalms):
    alms[j,:] = retrieve_alms(mode=j, nalms=nalms)
    
print(alms.shape)
print(alms[:10,:10])

(272, 272)
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


dtype('float64')

This doesn't really make much sense does it? I feel like there's still a problem in not having a proper reference time. 

# Tests that functions work

### Test of basic response function

In [26]:
# Reference observation time
obstime0 = Time('2022-06-27T01:02:30.11', format='isot', scale='utc') 

# Second observation time 
obstime1 = Time('2022-06-27T03:02:30.11', format='isot', scale='utc') 

# Time of rotation given as fraction of a sidereal day
hour_angle = np.array([(obstime1-obstime0).value])

# Test of array of times
hour_angles = np.linspace(0.2, 0.5, 2)# * units.hour


In [27]:
hour_angle[0]

0.08333333333333337

In [28]:
lmax = 15
nside = 64

bl = [14.,0.,0.]
freqs=np.linspace(100., 200., 20)

In [29]:
t0 = time.time()
v = vis_sh_response_multi_rotations(bl, freqs, lmax, hour_angles, nside)
print("Run took %5.3f sec" % (time.time() - t0))

Run took 78.558 sec


Doing:
```
test_array = np.zeros((4,3,2))
print(test_array)
```

will print:

```
[[[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]]
```

So to have dimensions of D_matrices x nmodes x freqs we need

`np.zeros((d_matrices.size, nmodes.size, freqs.size))`

In [30]:
real_v = split_complex_array(v)
print(real_v.shape)

(4, 272, 20)


### Speed up by parallelising?

Package, "parallel" to parallelise for loops

```
from joblib import Parallel, delayed
def process(i):
    return i * i
    
results = Parallel(n_jobs=2)(delayed(process)(i) for i in range(10))
print(results)  # prints [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
```

In [31]:
hour_angles = np.linspace(0, 0.5, 10)# fraction of a day

In [32]:
t0 = time.time()
v = vis_sh_response_multi_rotations(bl, freqs, lmax, hour_angles, nside)
for_loop_time = time.time() - t0
print("Run took %5.3f sec" % (for_loop_time))
print(v.shape)

KeyboardInterrupt: 

In [ ]:
t0 = time.time()
test_results = Parallel(n_jobs=-1)(delayed(vis_sh_response)(bl, freqs, lmax, hour_angles[i]) for i in range(hour_angles.size))
parallel_time = time.time() - t0
print("Run took %5.3f sec" % (parallel_time))
print(np.array(test_results).shape)

Parallelisation works well! But maybe I should stack it instead so it doesn't need to iterate? I'm not sure if that helps with the problem though.



In [ ]:
hour_angles = np.linspace(0.1, 0.5, 2)# fraction of a day

In [ ]:
t0 = time.time()
test_results = Parallel(n_jobs=-1)(delayed(vis_sh_response)(bl, freqs, lmax, hour_angles[i]) for i in range(hour_angles.size))
parallel_time = time.time() - t0
print("Run took %5.3f sec" % (parallel_time))
print(np.array(test_results).shape)

```
nalms = 272
alm size is = 272
mode size is = 256
healpy_modes size is = 136
```

In [ ]:
print(np.array(test_results).dtype)

### Test of Rotations

In [ ]:
# Reference observation time
obstime0 = Time('2022-06-27T01:02:30.11', format='isot', scale='utc') 

# Second observation time 
obstime1 = Time('2022-06-27T03:02:30.11', format='isot', scale='utc') 

#obstimes = obstime0 + np.linspace(0., 24., 100) * units.hour


# Time of rotation given as fraction of a sidereal day
ha = (obstime1-obstime0).value


In [ ]:
t0 = time.time()
# D_matrix = D_matrix_1D(modes=modes_init, lmax=lmax, ra=ra_zenith, dec=dec_zenith, ha=rotation_time)
D_matrix = D_matrix_1D(lmax, ha)
block_diag_D_matrix = combined_D_matrix(D_matrix, lmax)
print("Run took %5.3f sec" % (time.time() - t0))

In [ ]:
t0 = time.time()
for j in range(10):
    alms = create_alms(mode=j,lmax=lmax)
    modes = alms2modes(alms,lmax)
    rotated_modes = rotate_D_matrix(block_diag_D_matrix, modes, lmax)
    healpy_modes = modes2healpy(rotated_modes, lmax)
print("Run took %5.3f sec" % (time.time() - t0))

In [ ]:
t0 = time.time()
for j in range(10):
    alms = create_alms(mode=j,lmax=lmax)
    modes = alms2modes(alms,lmax)
    apply_D_matrix = np.dot(combined_D_matrix(D_matrix, lmax), modes)
    rotated_modes = spherical.Modes(apply_D_matrix, spin_weight=0)
    healpy_modes = modes2healpy(rotated_modes, lmax)
print("Run took %5.3f sec" % (time.time() - t0))

### Test of mode manipulation/reordering

In [241]:
# test functions works:
lmax, mode= (6, 15)

alms = create_alms(mode,lmax)
wigner_modes = alms2modes(alms, lmax)
alms_back = modes2alms(wigner_modes,lmax)

print(alms==alms_back)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]


In [ ]:
healpy_modes = modes2healpy(wigner_modes,lmax)

print(healpy_modes.real == alms[:int(np.size(alms)/2)])
print(healpy_modes.imag == alms[int(np.size(alms)/2):])

alms_back_again = healpy2alms(healpy_modes)

print(alms==alms_back_again)